In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/olivetti-faces/olivetti_faces_target.npy
/kaggle/input/olivetti-faces/olivetti_faces.npy


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

LR = 1e-3
epochs = 50
BATCH_SIZE = 20

olivetti_faces = fetch_olivetti_faces(data_home='./', shuffle=True)
X = olivetti_faces.data
y = olivetti_faces.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
x_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = [(img, lbl) for img, lbl in zip(x_train, y_train)]
test_dataset = [(img, lb1) for img, lb1 in zip(x_test, y_test)]
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [9]:
def create_model():
    model = nn.Sequential(
        nn.Linear(4096, 12288),
        nn.BatchNorm1d(12288),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(12288, 8192),
        nn.BatchNorm1d(8192),
        nn.ReLU(),
        nn.Dropout(0.6),
        nn.Linear(8192, 2048),
        nn.BatchNorm1d(2048),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(2048, 40)
    ).to(device)
    return model

In [10]:
model = create_model()

In [11]:

loss_fn = nn.CrossEntropyLoss()
optimizers = {
        'sgd': optim.SGD(model.parameters(), lr=LR, momentum=0.9),
        'adam': optim.Adam(model.parameters(), lr=LR),
        'rmsprop': optim.RMSprop(model.parameters(), lr=LR),
        'adagrad': optim.Adagrad(model.parameters(), lr=LR)
    }


loss_history = []
for optimizer_name, optimizer in optimizers.items():
    print(f"Using {optimizer_name} optimizer")
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = loss_fn(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_history.append(loss.item())
        print(f'Epoch {epoch} Loss: {loss.item()} with {optimizer_name}')
    







Using sgd optimizer
Epoch 0 Loss: 3.3289413452148438 with sgd
Epoch 1 Loss: 1.847638726234436 with sgd
Epoch 2 Loss: 1.195692777633667 with sgd
Epoch 3 Loss: 0.471047967672348 with sgd
Epoch 4 Loss: 0.5231450200080872 with sgd
Epoch 5 Loss: 0.29533660411834717 with sgd
Epoch 6 Loss: 0.3562431335449219 with sgd
Epoch 7 Loss: 0.20333795249462128 with sgd
Epoch 8 Loss: 0.15340350568294525 with sgd
Epoch 9 Loss: 0.09753087162971497 with sgd
Epoch 10 Loss: 0.16551896929740906 with sgd
Epoch 11 Loss: 0.14894574880599976 with sgd
Epoch 12 Loss: 0.06936604529619217 with sgd
Epoch 13 Loss: 0.11914755403995514 with sgd
Epoch 14 Loss: 0.07478485256433487 with sgd
Epoch 15 Loss: 0.05491945892572403 with sgd
Epoch 16 Loss: 0.05964955687522888 with sgd
Epoch 17 Loss: 0.058710455894470215 with sgd
Epoch 18 Loss: 0.08189171552658081 with sgd
Epoch 19 Loss: 0.07231154292821884 with sgd
Epoch 20 Loss: 0.08317961543798447 with sgd
Epoch 21 Loss: 0.02951253019273281 with sgd
Epoch 22 Loss: 0.0265109091997